# Variational Quantum Classifier (VQC) Splitting

This notebook will explore the Qiskit implementation of a VQC, and initial attempts to split the VQC into parts to explain.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [ ]:
# imports
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import clear_output

import os
import time

# import data class
from utilities.dataset_utils import DiabetesData

# import metrics for evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# qiskit imports
# circuit transpiler
from qiskit import transpile

# algorithm
from qiskit_machine_learning.algorithms.classifiers import VQC

# feature map
from qiskit.circuit.library import z_feature_map

# ansatz
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import EfficientSU2

# optimizer
from qiskit_machine_learning.optimizers import COBYLA

# simulator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

### VQC Overview
A Variational Quantum Classifier is a hybrid quantum-classical machine learning algorithm used for classification. It encodes the classical data using a feature map, which maps the data onto qubits to take advantage of quantum phenomena. The encoded data is processed through a variational quantum circuit (consists of parameterized gates). These parameters are iteratively optimised using classical algorithms to minimise a cost function (difference between predicted and true labels). After processing, quantum circuit is measured, and results are post processed to produce predictions. 
This hybrid approach allows the VQC allows the combination of the power of quantum mechanics with classical optimisation techniques, offering advantages for complex classification problems that would be challenging for a purely classical approach

In [ ]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [ ]:
# setup backend simulator
backend = AerSimulator()
backend.set_options(max_parallel_threads=os.cpu_count(), method='statevector')

In [ ]:
# feature map
feature_map = z_feature_map(feature_dimension=dataset.get_num_features(), reps=2, entanglement='linear')
feature_map = transpile(feature_map, backend=backend, optimization_level=3) # transpile to backend

In [ ]:
# ansatz
ansatz = EfficientSU2(num_qubits=dataset.get_num_features(), reps=3, entanglement='linear')
ansatz = transpile(ansatz, backend=backend, optimization_level=3) # transpile to backend

In [ ]:
# callback graph
# init list to store objective function values
objective_func_vals = []

# larger size for graph
plt.rcParams["figure.figsize"] = (12, 6)

# callback function to plot objective function value (updates after each iteration)
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective Function Value")
    plt.xlabel("Iteration")
    plt.ylabel("Objective Function Value")
    plt.plot(objective_func_vals)
    plt.show()

In [ ]:
# optimizer
optimizer = COBYLA(maxiter=500)

In [ ]:
# sampler
sampler = Sampler.from_backend(backend)

In [ ]:
# create the VQC instance
vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph
)

### Model Fit on Simulator

In [ ]:
# get training and testing data
X_train, X_test, y_train, y_test = dataset.preprocess_data()

In [1]:
# time how long it takes to train
start = time.time()

# fit the model
vqc = vqc.fit(X_train, y_train)

end = time.time()
elapsed = end - start

# print training time
print(f"Training Time: {round(elapsed)} secs")

# print objective func val at each iteration
print(objective_func_vals)

NameError: name 'time' is not defined

### VQC Evaluation

In [ ]:
# reset rcParams to default
plt.rcParams["figure.figsize"] = (6,4)

In [ ]:
train_score = vqc.score(X_train, y_train)
test_score = vqc.score(X_test, y_test)
train_score, test_score

In [ ]:
# predict
y_pred = vqc.predict(X_test)

In [ ]:
# confusion matrix
def conf_matrix(y_test, y_pred):
    plt.figure()
    cm = confusion_matrix(y_test, y_pred)
    heatmap = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
    plt.title('Confusion Matrix: VQC')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [ ]:
# accuracy score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# classification report
report = classification_report(y_test, y_pred, zero_division=0, output_dict=False)

In [ ]:
# print metrics
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')
conf_matrix(y_test, y_pred)

### Save Model

In [ ]:
# path to save model
model_path = os.path.join(os.getcwd(), '..', '..', 'models', 'qml-simulator')

# create model directory if it doesn't exist
os.makedirs(model_path, exist_ok=True)

# create model file
model_file = os.path.join(model_path, 'vqc_init.model')

# save model
vqc.save(model_file)
print(f'Model saved to {model_file}')

### Applying Explainability Techniques
##### **Pre-Prediciton**
1) **Feature Map**
    - Perturbation Analysis
    - Leave-One-Out (LOO) Analysis
    - Permutation Importance
    - SHAP (Feature Encoding)

    
##### **Post-Prediction**
1) **Ansatz (Variational Circuit)**
    - Parameter Sensitivity Analysis
    - Quantum Fisher Information (QFI)
    - Gradient-Based Analysis
    - Circuit Visualisation
    - SHAP (Parameters)
      - Applying importance scores to asses contribution to 
2) **Measurement + Post Processing**
    - SHAP (Predicitons)
    - ALE
    - Probability Attribution Analysis
3) **Cost**
    - Loss Contribution Analysis
    - Perturbation Sensitivity on Loss
    - Quantum-Classical Hybrid Loop Analysis